In [12]:
import pandas as pd
import numpy as np 
from itertools import product

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential 
from keras.layers import Dense, Dropout, GRU, Input
from keras.utils import pad_sequences
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [13]:
df = pd.read_csv('../data/asfp_cleaned_features.csv')

In [14]:
##############################
#### Hyperparameter Grid ####
##############################

feature_cols = ['water_temp', 'ph', 'ec', 'do', 'dt_seconds', 'segment', 'hour', 'dayofweek', 'month', 'water_temp_roll1h_mean', 'ph_roll1h_mean', 'ec_roll1h_mean', 'do_roll1h_mean']
input_features = ['water_temp', 'ph', 'ec', 'do', 'dt_seconds', 'hour', 'dayofweek', 'month']
output_features = ['water_temp', 'ph', 'ec', 'do']
n_inputs = len(input_features)
n_outputs = len(output_features)
pred_horizon = 6

param_grid = {
    'layer_units': [(64, 32), (128, 64)],
    'learning_rate': [0.001, 0.0025, 0.005],
    'dropout_rate': [0.05, 0.075, 0.1],
    'seq_length': [10, 15, 20],
}

keys = list(param_grid.keys())
combos = list(product(*param_grid.values()))
print(f"Total combinations per sensor: {len(combos)}")

for sensor_id in [1, 2]:
    print(f"\n{'='*60}")
    print(f"  SENSOR {sensor_id} - Grid Search")
    print(f"{'='*60}\n")

    #### Split & Scale ####
    df_sub = df[df['sensor'] == sensor_id]
    X_train, X_test = train_test_split(df_sub, test_size=0.2, shuffle=False)
    df_train = X_train.reset_index()
    df_test = X_test.reset_index()

    scaler = MinMaxScaler()
    df_train[feature_cols] = scaler.fit_transform(df_train[feature_cols])
    df_test[feature_cols] = scaler.transform(df_test[feature_cols])

    results = []

    for i, combo in enumerate(combos):
        params = dict(zip(keys, combo))
        l1, l2 = params['layer_units']
        lr = params['learning_rate']
        dr = params['dropout_rate']
        sl = params['seq_length']

        # Build training sequences for this seq_length
        seq_arrays = []
        seq_labels = []
        for j in range(len(df_train) - sl - pred_horizon):
            seq_arrays.append(df_train[input_features].iloc[j:j+sl].values)
            seq_labels.append(df_train[output_features].iloc[j+sl+pred_horizon].values)
        seq_arrays = np.array(seq_arrays)
        seq_labels = np.array(seq_labels)

        # Build test sequences for this seq_length
        test_arrays = []
        test_labels = []
        for j in range(2, len(df_test) - pred_horizon):
            start_idx = max(0, j - sl)
            test_arrays.append(df_test[input_features].iloc[start_idx:j].values)
            test_labels.append(df_test[output_features].iloc[j + pred_horizon].values)
        test_arrays = pad_sequences(test_arrays, maxlen=sl, dtype='float32', padding='pre')
        test_labels = np.array(test_labels, dtype=np.float32)

        # Build model
        model = Sequential()
        model.add(Input(shape=(sl, n_inputs)))
        model.add(GRU(units=l1, return_sequences=True))
        model.add(Dropout(dr))
        model.add(GRU(units=l2, return_sequences=False))
        model.add(Dropout(dr))
        model.add(Dense(units=n_outputs))
        model.compile(optimizer=Adam(lr), loss='mse', metrics=['mse'])

        # Train
        history = model.fit(
            seq_arrays, seq_labels,
            epochs=100, batch_size=500, validation_split=0.05, verbose=0,
            callbacks=[EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)]
        )

        # Evaluate
        scores = model.evaluate(test_arrays, test_labels, verbose=0)
        test_mse = scores[1]

        best_val = min(history.history['val_loss'])
        epochs_run = len(history.history['loss'])

        results.append({
            'layer_units': (l1, l2), 'learning_rate': lr, 'dropout_rate': dr,
            'seq_length': sl, 'test_mse': test_mse, 'best_val_mse': best_val,
            'epochs': epochs_run
        })

        print(f"[{i+1}/{len(combos)}] units=({l1},{l2}) lr={lr} dr={dr} seq={sl} -> test_mse={test_mse:.6f} (epochs={epochs_run})")

    # Summary
    results_df = pd.DataFrame(results).sort_values('test_mse')
    print(f"\n=== Sensor {sensor_id} - Top 10 Configurations ===")
    print(results_df.head(10).to_string(index=False))

Total combinations per sensor: 54

  SENSOR 1 - Grid Search

[1/54] units=(64,32) lr=0.001 dr=0.05 seq=10 -> test_mse=0.006142 (epochs=52)
[2/54] units=(64,32) lr=0.001 dr=0.05 seq=15 -> test_mse=0.006511 (epochs=20)
[3/54] units=(64,32) lr=0.001 dr=0.05 seq=20 -> test_mse=0.006220 (epochs=23)
[4/54] units=(64,32) lr=0.001 dr=0.075 seq=10 -> test_mse=0.006477 (epochs=35)
[5/54] units=(64,32) lr=0.001 dr=0.075 seq=15 -> test_mse=0.006355 (epochs=46)
[6/54] units=(64,32) lr=0.001 dr=0.075 seq=20 -> test_mse=0.006260 (epochs=60)
[7/54] units=(64,32) lr=0.001 dr=0.1 seq=10 -> test_mse=0.006565 (epochs=38)
[8/54] units=(64,32) lr=0.001 dr=0.1 seq=15 -> test_mse=0.006476 (epochs=36)
[9/54] units=(64,32) lr=0.001 dr=0.1 seq=20 -> test_mse=0.006678 (epochs=35)
[10/54] units=(64,32) lr=0.0025 dr=0.05 seq=10 -> test_mse=0.006232 (epochs=37)
[11/54] units=(64,32) lr=0.0025 dr=0.05 seq=15 -> test_mse=0.006059 (epochs=58)
[12/54] units=(64,32) lr=0.0025 dr=0.05 seq=20 -> test_mse=0.006479 (epochs=2